# 4. 캐글 경진대회 도전 (2) : 주택 가격 예측
## 4.1 더 상세하게 알아보는 데이터 분석
 - 데이터 분석 방법을 좀 더 상세히 알아보기
 - 여러 머신 러닝 방법의 설계 및 조합
 - 추가 분석 (1) : 클러스터 분석에 따른 주택 분류
 - 추가 분석 (2) : 고급 주택의 조건 분석 및 시각화
 
## 4.2 주택 가격 예측하기 경진대회란

예측 정확도 평가는 평균 제곱 오차(RMSE, Root Mean Squared Error)를 사용하며, 정답 데이터와 차이가 작을수록 상위를 차지합니다.


**분류(classification)**

|ID|Survived|
|-----|-----|
|0|1|
|1|1|
|2|0|
|3|1|
|4|0|

**회귀(regression)**

|ID|SalePrice|
|-----|-----|
|0|112,310|
|1|145,560|
|2|219,200|
|3|186,500|
|4|134,365|

평가 항목(evaluation)을 보면 Taking logs mean that errors in predicting expensive houses and cheap houses will affect the result equally

## 4.3 데이터 내려받기

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("ggplot")

In [2]:
import pandas as pd
import numpy as np

#### 랜덤 시드 설정하기

In [3]:
import random
np.random.seed(1234)
random.seed(1234)

#### CSV 데이터 불러오기(아나콘다(윈도), 맥에서 주피터 노트북 이용)

In [4]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")
submission = pd.read_csv("./data/sample_submission.csv")

In [5]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## 4.4 벤치마크용 베이스라인 작성

먼저 최소한의 카테고리 변수를 라벨 인코딩 처리한 후 LightGBM에 넣었을 때 정확도가 얼마인지 알아보겠습니다. 이것은 이후 처리가 정확도 향상에 얼마나 효과가 있었는지 검증하기 위해서 입니다.

### 4.4.1 LightGBM으로 예측

각 변수의 중요도를 나타낼 수 있기 때문에 향후 처리의 우선순위 등을 생각하는 지표로 활용될 수 있음

#### 학습 데이터의 변수 유형 확인하기
학습 데이터 각 변수의 유형을 **dtypes 명령어**로 확인할 수 있습니다

In [6]:
train_df.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [7]:
train_df["MSZoning"].value_counts()

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64